In [8]:
import numpy as np
import torch 
from torch.autograd.functional import jacobian
from torch.autograd import Variable


In [11]:
obj = lambda x: x[0]**2 + (x[1] - 3)**2
g1 = lambda x: x[1]**2 + 2*x[0]
g2 = lambda x: (x[1]-1)**2 + 5*x[0]-15
x = Variable(torch.tensor([1,1.]), requires_grad=True)

In [17]:
# Calculating Reduce Gradient using jacobian
def df(obj,g1,g2,x):
  #compute Jacobian
  Jocobian = torch.zeros((3, 2))
  Jocobian[0] = jacobian(obj, (x))
  Jocobian[1] = jacobian(g1, (x))
  Jocobian[2] = jacobian(g2, (x))
  return Jocobian 


In [18]:
Dfdd=df(obj,g1,g2,x)

In [19]:
Dfdd

tensor([[ 2., -4.],
        [ 2.,  2.],
        [ 5.,  0.]])

In [9]:
def lineSearch(x, t0=0.5, K=25):
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    alpha = 1
    i = 0
    func = f(newX(x, alpha))
    phi = f(x) - (t0 * alpha * (ReduceJacobian ** 2))
    while func > phi and i < K:
        alpha = 0.5 * alpha
        func = f(newX(x, alpha))
        phi = f(x)- (t0 * alpha * (ReduceJacobian ** 2))
        i += 1
    return alpha